
---
# Load needed libraries and functions

In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%run ../py/html_analysis.py
hc = HeaderCategories()
ha = HeaderAnalysis()
ea = ElementAnalysis()

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\CS_CV.pickle
Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\CHILD_STR_CLF.pickle



---
# Training

In [117]:

# Rebuild the datframe from the dictionary
rows_list = [{'qualification_str': qualification_str, 'is_fit': is_fit} for qualification_str, is_fit in NAVIGABLE_PARENT_IS_QUAL_DICT.items()]
NAVIGABLE_PARENT_IS_QUAL_DF = pd.DataFrame(rows_list)
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DF=NAVIGABLE_PARENT_IS_QUAL_DF)

# Re-transform the bag-of-words and tf-idf from the new manual scores
sents_list = NAVIGABLE_PARENT_IS_QUAL_DF.qualification_str.tolist()

# Bag-of-words
bow_matrix = ea.CS_CV.fit_transform(sents_list)
s.store_objects(BQ_CV_VOCAB=ea.CS_CV.vocabulary_)

# Tf-idf, must get from BOW first
tfidf_matrix = ea.CS_TT.fit_transform(bow_matrix)
s.store_objects(BQ_TT=ea.CS_TT)

# Re-train the classifier
X = tfidf_matrix.toarray()
y = NAVIGABLE_PARENT_IS_QUAL_DF.is_fit.to_numpy()
BASIC_QUALS_CLF = ea.FIT_ESTIMATORS_DICT['LogisticRegression']
BASIC_QUALS_CLF.fit(X, y)
s.store_objects(BASIC_QUALS_CLF=BASIC_QUALS_CLF)

# Re-calibrate the inference engine
BQ_CV_VOCAB = s.load_object('BQ_CV_VOCAB')
bq_cv = CountVectorizer(vocabulary=BQ_CV_VOCAB)
bq_cv._validate_vocabulary()
BQ_TT = s.load_object('BQ_TT')
def predict_is_qual_percent_fit(quals_list):
    y_predict_proba = np.array([])
    if len(quals_list):
        X_test = BQ_TT.transform(bq_cv.transform(quals_list)).toarray()
        y_predict_proba = BASIC_QUALS_CLF.predict_proba(X_test)
    
    return y_predict_proba
print('Retraining complete')

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\NAVIGABLE_PARENT_IS_QUAL_DF.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\BQ_CV_VOCAB.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\BQ_TT.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\BASIC_QUALS_CLF.pickle
Retraining complete



---
# Inference

In [119]:

# Loop through all the unset %fit values, set them if you can, break for help if you can't
mask_series = (hunting_df.percent_fit >= 0.0)
for row_index, row_series in hunting_df[~mask_series].iterrows():
    quals_list, job_fitness = print_fit_job(row_index, row_series, NAVIGABLE_PARENT_IS_QUAL_DICT)
    if job_fitness > 0.8:
        if all(qual_str in NAVIGABLE_PARENT_IS_QUAL_DICT for qual_str in quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
        else:
            break
    else:
        if len(quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
print('{}/{} = {}% completed'.format(hunting_df[mask_series].shape[0], hunting_df.shape[0],
                                     int(100 * hunting_df[mask_series].shape[0] / hunting_df.shape[0])))

5513/5513 = 100% completed


In [116]:

# Manually label the unscored qual
qualification_str = quals_list[2]
print(qualification_str)
NAVIGABLE_PARENT_IS_QUAL_DICT[qualification_str] = 0
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DICT=NAVIGABLE_PARENT_IS_QUAL_DICT)

Experience with WordPress, Joomla, or Microsoft SharePoint
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\NAVIGABLE_PARENT_IS_QUAL_DICT.pickle



---
# Create the emails
Can you work in automation to ensure you are not emailing the same employee more than once during a 2 week span?

In [120]:

def f(x):
    
    return 'python' in str(x).lower()

def g(x):
    
    return 'polygraph' in str(x).lower()

ds_regex = re.compile(r'data *scien')
ml_regex = re.compile(r'machine *learning|\bML\b')
def ff(x):
    x = str(x).lower()
    if ds_regex.search(x) or ml_regex.search(x) or ('python' in x):
        match = True
    else:
        match = False
    
    return match

mask_series = hunting_df['Job Requisition Type'].isin(['Sold and Funded', 'Sold & Unfunded'])
#mask_series = mask_series & hunting_df['Job Description'].map(f)
mask_series = mask_series & hunting_df['Job Description'].map(ff)
mask_series = mask_series & (hunting_df.percent_fit >= 0.85) & ~hunting_df['is_opportunity_application_emailed']
mask_series = mask_series & ~(hunting_df.is_remote_delivery == False)
mask_series = mask_series & ~hunting_df['Required Clearance'].isin(['TS/SCI', 'TS/SCI w/CIP', 'TS/SCI w/FSP'])
mask_series = mask_series & ~hunting_df['Job Description'].map(g)
print(hunting_df[mask_series].shape)
print(hunting_df[mask_series].groupby('Required Clearance').count().T.max().sort_values(ascending=False))

(3, 37)
Required Clearance
Secret    3
dtype: int64


In [121]:

hunting_df[mask_series].head(5).T

,5213,5279,5324
Job Posting,Data Analyst,Financial Intelligence Data Scientist,Immersive Analytics Data Scientist
Job Requisition,R0082190 Data Analyst (Open),R0082323 Financial Intelligence Data Scientist...,R0081829 Immersive Analytics Data Scientist (O...
Job Requisition ID,R0082190,R0082323,R0081829
Job Requisition Status,Open,Open,Open
Furthest Stage,Review,Review,Review
Supervisory Organization,Supervisory Organization (Kasey Bushmire (5572...,Supervisory Organization (James FitzGerald | J...,Supervisory Organization (Kenneth Brodeur (599...
Group,GLOBAL DEFENSE GROUP,NATIONAL SECURITY GROUP,STRATEGIC INNOVATION GROUP
Account Group,JCC ACCT GROUP,NATL AGENCIES ACCT GROUP,FSO ACCT GROUP
IMT,JCC IMT,NA ANALYTICS IMT,ANALYTICS IMT
Cluster,Wash Metro Cluster,Wash Metro Cluster,Huntsville Cluster


In [123]:

from datetime import datetime

email_tuples_list = print_emails(mask_series)

In [ ]:

# Manually note the email has been sent
req_id = 'R0077973'.strip()
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
date_obj = datetime.now()
hunting_df.loc[mask_series, 'opportunity_application_email_date'] = pd.Timestamp(date_obj)
s.store_objects(hunting_df=hunting_df)
last_emailed_dict = s.load_object('last_emailed_dict')
row_series = hunting_df.loc[mask_series].iloc[0]
email_tuple = get_email_tuple(row_series)
for email in email_tuple:
    last_emailed_dict[email] = pd.Timestamp(date_obj)
s.store_objects(last_emailed_dict=last_emailed_dict)


---
# Flag setting

In [ ]:

# Set notes for this one job
req_id = 'R0073507'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
manager_notes = hunting_df.loc[mask_series, 'manager_notes'].tolist()[0]
if len(manager_notes):
    manager_notes += ' '
manager_notes += 'The position is put on hold right now.'
manager_notes += ''
hunting_df.loc[mask_series, 'manager_notes'] = manager_notes
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Set remote delivery for this one job
req_id = 'R0073507'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)